<a href="https://colab.research.google.com/github/beingraja/Keras-GAN/blob/master/face_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jan 27 18:08:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset

 99% 1.32G/1.33G [00:25<00:00, 55.3MB/s]
100% 1.33G/1.33G [00:25<00:00, 55.1MB/s]


In [ ]:
!unzip /content/celeba-dataset.zip

In [ ]:
import os
images = os.listdir('/content/img_align_celeba/img_align_celeba')

In [ ]:
images = images[50000:75000]

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tqdm import tqdm_notebook
import numpy as np
import tqdm
img_shape = (64, 64, 3)
data_dir = '/content/img_align_celeba/img_align_celeba'
def get_data(data_path) :
    X = []
    for filename in tqdm_notebook(data_path) :
        img = img_to_array(load_img(data_dir + "/" + filename, target_size = img_shape[:2]))
        X.append(img)
    X = np.array(X).astype('float32')
    #X = (X - 127.5) / 127.5
    X = X / 255
    return X

X = get_data(images)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [ ]:
i = int(input('hello bitch'))
plt.imshow(X[i])

KeyboardInterrupt: ignored

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense , Reshape
from tensorflow.keras.layers import Conv2D , Conv2DTranspose
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
from numpy.random import randn
from numpy.random import randint
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import cv2

In [ ]:
def discriminator(in_shape = (64,64,3)):
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(2,2),padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(128,(3,3),strides=(2,2),padding='same'))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128,(3,3),strides=(2,2),padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()
    return model

discriminator()  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0

In [ ]:
def generator(latent_dim):
    n = 8 * 8* 128
    model = Sequential()
    model.add(Dense(n, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))  

    model.add(Reshape((8, 8, 128)))
    model.add(Conv2DTranspose(128, (3,3), strides=(2,2), padding='same'))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    #model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(3, (7,7), activation='tanh', padding='same'))
    model.summary()
    return model 
generator(100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 8192)              0         
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 128)       147584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 128)       147584    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 32, 128)      

In [ ]:
def gan(discriminator , generator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    model.summary()
    discriminator.trainable = True
    return model


In [ ]:
import os 
def generate_latent(batch_size = 128,latent_dim=100):
    latent = randn(latent_dim*batch_size)
    latent = np.reshape(latent,(batch_size,latent_dim))
    return latent
def generator_fake_samples(gen,latent_dim,batch_size = 128):
    lat = generate_latent(batch_size=128,latent_dim=100)
    X = gen.predict(lat)
    y = np.zeros((batch_size,1))
    return X,y
def generate_real(data , batch_size = 128):
    rand = randint(0,data.shape[0],batch_size)
    X = data[rand]
    y = np.ones((batch_size,1))
    return X,y
def save_images(gen,epoch,latent_dim,noise,size = 5):

    gen_imgs = gen.predict(noise)
    
    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    path = '/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images'
    try:
        os.mkdir('/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images')
    except FileExistsError:
        pass
    fig, axs = plt.subplots(size, size)
    cnt = 0
    for i in range(size):
        for j in range(size):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images/images_%d.png" % epoch)
    plt.close()


def save_model(gen,dis,combo,i,iv):

    try:
        os.mkdir('/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/models')
    except FileExistsError:
        pass
    path = '/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/'
    gen.save(path+'models/generator_epoch_'+ str(i)+'.h5') 
    dis.save(path+'models/discriminator_epoch_'+ str(i)+'.h5')   
    combo.save(path+'models/gan_epoch_'+ str(i)+'.h5')     

    if i > iv:
        with open(path+'models/generator_epoch_'+ str(i-iv)+'.h5','w') as f:
            f.close()
        os.remove(path+'models/generator_epoch_'+ str(i-iv)+'.h5')
        with open(path+'models/discriminator_epoch_'+ str(i-iv)+'.h5','w') as f:
            f.close() 
        os.remove(path+'models/discriminator_epoch_'+ str(i-iv)+'.h5')
        with open(path+'models/gan_epoch_'+ str(i-iv)+'.h5','w') as f:
            f.close()
        os.remove(path+'models/gan_epoch_'+ str(i-iv)+'.h5')

In [ ]:
def train( gen , dis, combo , data, model_save_interval, generated_images_interval ,epoch = 2 , batch_size = 128 ):
    noise = np.random.normal(0, 1, (5 * 5, 100))
    for i in range(epoch):
        n_batch =int(data.shape[0] / batch_size)
        for j in range(n_batch):
            X_real,y_real = generate_real(data,batch_size=128)
            X_fake,y_fake = generator_fake_samples(gen,100,batch_size)

            #discriminator loss
            dfake_loss , _ = dis.train_on_batch(X_fake,y_fake)
            dreal_loss, _ = dis.train_on_batch(X_real,y_real)
            d_loss = 0.5*np.add(dfake_loss,dreal_loss)

            #for combined gan 
            latent = generate_latent(batch_size=128,latent_dim=100)
            y_combo = np.ones((batch_size,1))
            gan_loss = combo.train_on_batch(latent,y_combo)
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i, j, n_batch, d_loss, gan_loss))

        if i != 0 and i != 1 and i % model_save_interval == 0:
            save_model(gen,dis,combo,i,model_save_interval)
        if  i % generated_images_interval==0:
            save_images(gen,i,100,noise,5)
    return gen

In [ ]:
from tensorflow.keras.models import load_model
def run_model(load_saved = False):    
    if load_saved == True:
        i = 95
        path = '/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/'
        gen = load_model(path+'models/generator_epoch_'+ str(i)+'.h5')
        dis = load_model(path+'models/discriminator_epoch_'+ str(i)+'.h5')
        dis.trainable = True
        combo = gan(dis,gen)
        dis.summary()
        gen= train(gen,dis,combo,X,5,100)
    else:
        gen = generator(100)
        dis = discriminator()
        combo = gan(dis,gen)
        gen = train(gen,dis,combo,X,5,1,epoch=100)
    return gen
 
gen = run_model(True)

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import numpy as np
noise = np.random.normal(0, 1, (1, 100))
gen = load_model('/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/models/generator_epoch_95.h5')
gen_imgs = gen.predict(noise)
# Rescale images 0 - 1
gen_imgs = np.array(0.5 * gen_imgs + 0.5).reshape((64,64,3))

plt.imshow(gen_imgs)

In [ ]:
def save_images():

    gen_imgs = gen.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    path = '/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images'
    try:
        os.mkdir('/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images')
    except FileExistsError:
        pass
    fig, axs = plt.subplots((size, size))
    cnt = 0
    for i in range(size):
        for j in range(size):
            axs[i,j].imshow(cv2.cvtColor(gen_imgs[cnt, :,:,0],cv2.COLOR_BGR2RGB))
            axs[i,j].axis('off')
            cnt += 1
    
    fig.savefig("/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images/images_%d.png" % epoch)
    plt.close()

In [ ]:
import os
l = os.listdir('/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images')

In [ ]:
import os
import imageio

png_dir ='/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/images'
images = []
for file_name in os.listdir(png_dir):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave('movie.gif', images)

In [ ]:
!!pip install tensorflow==2.1


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
i = 95
path = '/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/'
gen = load_model(path+'models/generator_epoch_'+ str(i)+'.h5')
dis = load_model(path+'models/discriminator_epoch_'+ str(i)+'.h5')
combo = load_model(path+'models/gan_epoch_'+ str(i)+'.h5')

In [ ]:
i = 95
path = '/content/drive/MyDrive/Colab Notebooks/Untitled Folder/face_gan/'
gen = load_model(path+'models/generator_epoch_'+ str(i)+'.h5')
dis = load_model(path+'models/discriminator_epoch_'+ str(i)+'.h5')
combo = load_model(path+'models/gan_epoch_'+ str(i)+'.h5')

In [ ]:

dis.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu_98 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_42 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_99 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_100 (LeakyReLU)  (None, 8, 8, 128)       